In [83]:
import sys

sys.path.append('../src')
from infrastructure.postgres_db import PostgresDB
from infrastructure.gpt4_chat import GPT4Chat
from application.services.sqlcodermodel import GPT4SQLModel

from dotenv import load_dotenv

load_dotenv()

db = PostgresDB("elecciones_chile", "jm", "")
gpt4 = GPT4Chat()
gpt4_sql =GPT4SQLModel(gpt4, db.get_schema())



In [9]:
query = gpt4_sql.to_sql("Participación en elecciones presidenciales primera vuelta de 2013")

In [22]:
query2 = "\n".join(query.splitlines()[1:-1])

In [25]:
print(query2)

SELECT 
    e.nombre AS election_name,
    e.fecha AS election_date,
    c.nombre AS candidate_name,
    p.nombre AS party_name,
    SUM(v.votos) AS total_votes
FROM 
    eleccion e
JOIN 
    candidatura cd ON e.id = cd.eleccion_id
JOIN 
    candidato c ON cd.candidato_id = c.id
JOIN 
    partido p ON cd.partido_id = p.id
JOIN 
    votos_mesa v ON cd.id = v.candidatura_id
WHERE 
    e.nombre = 'Presidenciales Primera Vuelta 2013'
GROUP BY 
    e.nombre, e.fecha, c.nombre, p.nombre
ORDER BY 
    total_votes DESC;


In [86]:
from scipy import spatial
from application.services.embeddings.openai_embedding import OpenAIEmbedding


embedding = OpenAIEmbedding() 

emb2 = embedding.generate("hola")
emb1 = embedding.generate("chao")
1 - spatial.distance.cosine(emb1, emb2).item()


0.42213860500562195

In [89]:
from application.services.cosine_similarity import CosineSimilarity

x = [
    "Plebiscito Constitucional 2022",
    "Presidencial 2021 Primera Vuelta",
    "Presidencial 2021 Segunda Vuelta",
    "Municipal 2021 - Alcaldes",
    "Municipal 2021 - Concejales",
    "Municipal 2016 - Alcaldes",
    "Municipal 2016 - Concejales",
    "Municipal 2012 - Alcaldes",
    "Municipal 2012 - Concejales",
    "CORE 2021",
    "CORE 2017",
    "CORE 2013",
    "Plebiscito Constitucional 2020 Apruebo Rechazo",
    "Presidencial 2017 Primera Vuelta",
    "Presidencial 2017 Segunda Vuelta",
    "Presidencial 2013 Primera Vuelta",
    "Presidencial 2013 Segunda Vuelta",
    "Senadores 2021",
    "Senadores 2017",
    "Senadores 2013",
    "Diputados 2021",
    "Diputados 2017",
    "Diputados 2013"
]
cs = CosineSimilarity(embedding=embedding)
# for j in x:
#     cs("Presidenciales Primera Vuelta 2013", )

In [94]:
import pandas as pd


x1 = pd.DataFrame(x, columns=["name"])
x1["similarity"] = x1["name"].apply(lambda x: cs("Presidenciales Primera Vuelta 2013", x))

In [96]:
x1.sort_values("similarity", ascending=False)

,name,similarity
15,Presidencial 2013 Primera Vuelta,0.925457
16,Presidencial 2013 Segunda Vuelta,0.893589
13,Presidencial 2017 Primera Vuelta,0.862861
14,Presidencial 2017 Segunda Vuelta,0.836098
1,Presidencial 2021 Primera Vuelta,0.818734
2,Presidencial 2021 Segunda Vuelta,0.788672
22,Diputados 2013,0.581259
19,Senadores 2013,0.498277
21,Diputados 2017,0.484960
8,Municipal 2012 - Concejales,0.454102


In [42]:
from sqlglot import parse_one, exp

# print all column references (a and b)
# for column in parse_one(query2).find_all(exp.Column):
#     print(column.alias_or_name)

# find all projections in select statements (a and c)
# for select in parse_one(query2).find_all(exp.Select):
#     for projection in select.expressions:
#         print(projection.alias_or_name)

# find all tables (x, y, z)
for table in parse_one(query2).find_all(exp.Where):
    print(table.to_s())

Where(
  this=EQ(
    this=Column(
      this=Identifier(this=nombre, quoted=False, _type=None, _comments=None, _id=4774326096),
      table=Identifier(this=e, quoted=False, _type=None, _comments=None, _id=4774325584),
      db=None,
      catalog=None,
      _type=None,
      _comments=None,
      _id=4774326352),
    expression=Literal(this=Presidenciales Primera Vuelta 2013, is_string=True, _type=None, _comments=None, _id=4774316624),
    _type=None,
    _comments=None,
    _id=4774315728),
  _type=None,
  _comments=None,
  _id=4774318800)


In [47]:
print(repr(parse_one(query2)))

Select(
  expressions=[
    Alias(
      this=Column(
        this=Identifier(this=nombre, quoted=False),
        table=Identifier(this=e, quoted=False)),
      alias=Identifier(this=election_name, quoted=False)),
    Alias(
      this=Column(
        this=Identifier(this=fecha, quoted=False),
        table=Identifier(this=e, quoted=False)),
      alias=Identifier(this=election_date, quoted=False)),
    Alias(
      this=Column(
        this=Identifier(this=nombre, quoted=False),
        table=Identifier(this=c, quoted=False)),
      alias=Identifier(this=candidate_name, quoted=False)),
    Alias(
      this=Column(
        this=Identifier(this=nombre, quoted=False),
        table=Identifier(this=p, quoted=False)),
      alias=Identifier(this=party_name, quoted=False)),
    Alias(
      this=Sum(
        this=Column(
          this=Identifier(this=votos, quoted=False),
          table=Identifier(this=v, quoted=False))),
      alias=Identifier(this=total_votes, quoted=False))],
  from=F

In [ ]:
for 

In [64]:
eqs = list(parse_one(query2).find(exp.Where).find_all(exp.EQ))

In [97]:
eqs

[EQ(
   this=Column(
     this=Identifier(this=nombre, quoted=False),
     table=Identifier(this=e, quoted=False)),
   expression=Literal(this=Presidenciales Primera Vuelta 2013, is_string=True))]

In [72]:
parse_one(query2).find(exp.Where)

Where(
  this=EQ(
    this=Column(
      this=Identifier(this=nombre, quoted=False),
      table=Identifier(this=e, quoted=False)),
    expression=Literal(this=Presidenciales Primera Vuelta 2013, is_string=True)))

In [76]:
p1 = parse_one(query2)
table_alias = {t.alias:t.name for t in p1.find_all(exp.Table)}

In [79]:
table_alias

{'e': 'eleccion',
 'cd': 'candidatura',
 'c': 'candidato',
 'p': 'partido',
 'v': 'votos_mesa'}

In [110]:
eqs[0].find(exp.Column).table

'e'

In [ ]:
from sqlalchemy import MetaData


metadata = MetaData()
metadata.reflect(bind=db.engine)
schema = metadata.schema


In [ ]:
metadata.tables["candidato"]

In [ ]:
db.engine.conn
